Welcome to quickstart notebook of ResNet-RS Keras package.

We will go over some basic concepts, like 
1. Installation.
2. Download data + fine tune.
3. Convert to TFLite.
4. Convert to ONNX.

Execute the cell below to check if we are using a GPU:

In [1]:
!nvidia-smi

Thu Nov 11 10:57:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Installation

Run below cell to install the module:

In [2]:
!pip install -q git+https://github.com/sebastian-sz/resnet-rs-keras@main

In [3]:
import os

import tensorflow as tf
from resnet_rs import ResNetRS50


print(tf.__version__)

2.7.0


### Download example dataset

In this section we are going to download example dataset.

In [4]:
!curl https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz | tar xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  218M  100  218M    0     0  65.0M      0  0:00:03  0:00:03 --:--:-- 65.0M


Remove the License file so it doesn't mess up directory structure:

In [5]:
!rm flower_photos/LICENSE.txt

Preview Class names:

In [6]:
!ls flower_photos

daisy  dandelion  roses  sunflowers  tulips


### Load the data:

In [10]:
DATA_PATH = "./flower_photos"
BATCH_SIZE = 32
TARGET_SIZE = (224, 224)


# Preprocess with Imagenet's mean and stddev:
def preprocess(image): 
    image = image / 255.
    image -= tf.constant([0.485, 0.456, 0.406], shape=[1, 1, 3], dtype=image.dtype)
    image /= tf.constant([0.229, 0.224, 0.225], shape=[1, 1, 3], dtype=image.dtype)
    return image


def preprocess_data(images, labels):
    images = preprocess(images)
    return images, labels


def augment_data(images, labels):
    return tf.image.random_flip_left_right(images), labels

In [11]:
# Create tf.data.dataset objects:

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=DATA_PATH,
    batch_size=BATCH_SIZE,
    image_size=TARGET_SIZE,
    label_mode="categorical",
    seed=1234,
    validation_split=0.2,
    subset="training"
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=DATA_PATH,
    batch_size=BATCH_SIZE,
    image_size=TARGET_SIZE,
    label_mode="categorical",
    seed=1234,
    validation_split=0.2,
    subset="validation"
)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [12]:
# Apply preprocessing and augmentation:

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.map(preprocess_data, num_parallel_calls=AUTOTUNE).map(augment_data, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
val_dataset = val_dataset.map(preprocess_data, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

In [13]:
# Sanity check our dataset

for image_batch, label_batch in train_dataset.take(1):
    print(image_batch.shape)
    print(label_batch.shape)

(32, 224, 224, 3)
(32, 5)


### Train (extract features)

Let us fine tune Efficient Net Lite.

In [14]:
def build_model(num_classes=5):
    base_model = ResNetRS50(
        input_shape=(TARGET_SIZE[0], TARGET_SIZE[1], 3),
        include_top=False,
        pooling="avg",
        weights="imagenet"
    )

    base_model.trainable=False

    return tf.keras.Sequential([
        base_model,
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(num_classes, activation="softmax")
    ])

In [15]:
model = build_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

model.summary()

135372800/135360144 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet-rs-50 (Functional)   (None, 2048)              33696288  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 5)                 10245     
                                                                 
Total params: 33,706,533
Trainable params: 10,245
Non-trainable params: 33,696,288
_________________________________________________________________


In [16]:
model.fit(
    train_dataset,
    epochs=5,
    validation_data=val_dataset,
)

Epoch 1/5
92/92 [==============================] - 75s 411ms/step - loss: 0.6009 - accuracy: 0.7977 - val_loss: 0.3249 - val_accuracy: 0.9046
Epoch 2/5
92/92 [==============================] - 33s 350ms/step - loss: 0.2775 - accuracy: 0.9176 - val_loss: 0.2461 - val_accuracy: 0.9251
Epoch 3/5
92/92 [==============================] - 33s 350ms/step - loss: 0.2080 - accuracy: 0.9353 - val_loss: 0.2179 - val_accuracy: 0.9332
Epoch 4/5
92/92 [==============================] - 33s 350ms/step - loss: 0.1749 - accuracy: 0.9489 - val_loss: 0.2135 - val_accuracy: 0.9292
Epoch 5/5
92/92 [==============================] - 33s 350ms/step - loss: 0.1460 - accuracy: 0.9571 - val_loss: 0.1872 - val_accuracy: 0.9387


### Convert TFLite

We can convert the modified model to Tensorflow Lite:

In [18]:
# Convert
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save
with open("resnet_rs.tflite", "wb") as file:
  file.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpl7ggq3fz/assets


INFO:tensorflow:Assets written to: /tmp/tmpl7ggq3fz/assets


In [19]:
!ls *.tflite

resnet_rs.tflite


### Convert onnx

We can also convert this model to ONNX via `tf2onnx` package:

In [20]:
!pip install tf2onnx~=1.8.4

     |████████████████████████████████| 370 kB 5.4 MB/s 
     |████████████████████████████████| 12.7 MB 88 kB/s 
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0


In [21]:
# Save the model in TF's Saved Model format:

In [22]:
model.save("my_saved_model/")

INFO:tensorflow:Assets written to: my_saved_model/assets


INFO:tensorflow:Assets written to: my_saved_model/assets
/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [23]:
# Convert:
!python -m tf2onnx.convert \
  --saved-model my_saved_model/ \
  --output resnet_rs.onnx

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-11-11 11:05:38,118 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2021-11-11 11:05:53,179 - INFO - Signatures found in model: [serving_default].
2021-11-11 11:05:53,179 - WARNING - '--signature_def' not specified, using first signature: serving_default
2021-11-11 11:05:53,180 - INFO - Output names: ['dense']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-11-11 11:05:59,313 - WARNING - From /usr/local/lib/python3.7/dist-packages/tf2onnx/tf_loader.py:603: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-11-11 11:06:03,826 - INFO - Using tenso

In [24]:
!ls *.onnx

resnet_rs.onnx
